# Setup Comprehend Through CLI/API:
https://docs.aws.amazon.com/comprehend/latest/dg/get-started-customclass.html

# Setup Comprehend Through AWS Console 

https://docs.aws.amazon.com/comprehend/latest/dg/getting-started-document-classification.html

Good example of using Comprehend for Positive/Negative Sentiment:  https://github.com/aws-samples/amazon-comprehend-custom-entity/blob/master/3-AWS-Comprehend-Negative-Custom-Classifier.ipynb

# Make sure this SageMakerNotebookExecutionRole has access to Comprehend

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
comprehend = boto3.client('comprehend')

In [3]:
%store -r noheader_train_s3_uri

print(noheader_train_s3_uri)

s3://sagemaker-us-west-2-853609162619/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv


In [4]:
!aws s3 ls $noheader_train_s3_uri

2020-05-23 20:48:53   13697426 amazon_reviews_us_Digital_Software_v1_00_noheader.csv


# Create Data Access Role for Comprehend

## Create Policy

In [5]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "comprehend.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
} 

## Create Role and Attach Policies

In [6]:
iam_comprehend_role_name = 'DSOAWS_Comprehend'

In [7]:
import json
import boto3
from botocore.exceptions import ClientError

try:
    iam = boto3.client('iam')

    iam_role_comprehend = iam.create_role(
        RoleName=iam_comprehend_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS Comprehend Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        iam_role_comprehend = iam.get_role(RoleName=iam_comprehend_role_name)
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)

Role already exists


In [8]:
comprehend_s3_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:GetObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(bucket)
            ],
            "Effect": "Allow"
        }
    ]
}

print(comprehend_s3_policy_doc)


{'Version': '2012-10-17', 'Statement': [{'Action': ['s3:GetObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-west-2-853609162619/*'], 'Effect': 'Allow'}, {'Action': ['s3:ListBucket'], 'Resource': ['arn:aws:s3:::sagemaker-us-west-2-853609162619'], 'Effect': 'Allow'}, {'Action': ['s3:PutObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-west-2-853609162619/*'], 'Effect': 'Allow'}]}


# Attach Policy to Role

In [9]:
response = iam.put_role_policy(
    RoleName=iam_comprehend_role_name,
    PolicyName='DSOAWS_ComprehendPolicyToS3',
    PolicyDocument=json.dumps(comprehend_s3_policy_doc)
)

# Train the Model

In [10]:
prefix = 'models'

s3_output_job = 's3://{}/{}/{}'.format(bucket, prefix, 'comprehend/output')
print(s3_output_job)

s3://sagemaker-us-west-2-853609162619/models/comprehend/output


In [11]:
iam_role_comprehend_arn = iam_role_comprehend['Role']['Arn']

In [12]:
import datetime

id = str(datetime.datetime.now().strftime("%s"))

training_job = comprehend.create_document_classifier(
    DocumentClassifierName='Amazon-Customer-Reviews-Classifier-'+ id,
    DataAccessRoleArn=iam_role_comprehend_arn,
    InputDataConfig={
        'S3Uri': noheader_train_s3_uri
    },
    OutputDataConfig={
        'S3Uri': s3_output_job
    },
    LanguageCode='en'
)

In [13]:
import time

jobArn = training_job['DocumentClassifierArn']

max_time = time.time() + 3 * 60 * 60 # 3 hours
while time.time() < max_time:
    describe_custom_classifier = comprehend.describe_document_classifier(
        DocumentClassifierArn = jobArn
    )
    status = describe_custom_classifier["DocumentClassifierProperties"]["Status"]
    print("Custom classifier: {}".format(status))
    
    if status == "TRAINED" or status == "IN_ERROR":
        print('')
        print('Status {}'.format(status))
        print('')
        print(describe_custom_classifier["DocumentClassifierProperties"])
        break
        
    time.sleep(5)

Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: T

Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: T

# _Please Wait Until the ^^ Classifier ^^ is Trained Above._

In [14]:
!aws s3 ls 

2020-05-23 20:56:46 cloudtrail-dsoaws-853609162619
2020-05-23 20:56:43 dsoaws-data-upload-853609162619
2020-05-23 15:41:35 sagemaker-us-west-2-853609162619


# Show Results of the Classifier

In [15]:
print(describe_custom_classifier["DocumentClassifierProperties"])

{'DocumentClassifierArn': 'arn:aws:comprehend:us-west-2:853609162619:document-classifier/Amazon-Customer-Reviews-Classifier-1590268028', 'LanguageCode': 'en', 'Status': 'TRAINED', 'SubmitTime': datetime.datetime(2020, 5, 23, 21, 7, 8, 940000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2020, 5, 23, 21, 51, 42, 665000, tzinfo=tzlocal()), 'TrainingStartTime': datetime.datetime(2020, 5, 23, 21, 11, 24, 920000, tzinfo=tzlocal()), 'TrainingEndTime': datetime.datetime(2020, 5, 23, 21, 51, 22, 998000, tzinfo=tzlocal()), 'InputDataConfig': {'S3Uri': 's3://sagemaker-us-west-2-853609162619/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv'}, 'OutputDataConfig': {'S3Uri': 's3://sagemaker-us-west-2-853609162619/models/comprehend/output/853609162619-CLR-ce1b7084b897a9bba47e1b0ef12b779a/output/output.tar.gz'}, 'ClassifierMetadata': {'NumberOfLabels': 5, 'NumberOfTrainedDocuments': 27905, 'NumberOfTestDocuments': 3100, 'EvaluationMetrics': {'Accuracy': 0.5274, 'Precision': 0.5223, 'Recal

In [16]:
model_arn = describe_custom_classifier["DocumentClassifierProperties"]["DocumentClassifierArn"]
print(model_arn)

arn:aws:comprehend:us-west-2:853609162619:document-classifier/Amazon-Customer-Reviews-Classifier-1590268028


In [17]:
import os
#Retrieve the S3URI from the model output and create jobkey variable.
job_output = describe_custom_classifier["DocumentClassifierProperties"]["OutputDataConfig"]["S3Uri"]
print(job_output)

path_prefix = 's3://{}/'.format(bucket)

job_key = os.path.relpath(job_output, path_prefix)

print(job_key)

s3://sagemaker-us-west-2-853609162619/models/comprehend/output/853609162619-CLR-ce1b7084b897a9bba47e1b0ef12b779a/output/output.tar.gz
models/comprehend/output/853609162619-CLR-ce1b7084b897a9bba47e1b0ef12b779a/output/output.tar.gz


In [18]:
!aws s3 cp $job_output ./output.tar.gz


download: s3://sagemaker-us-west-2-853609162619/models/comprehend/output/853609162619-CLR-ce1b7084b897a9bba47e1b0ef12b779a/output/output.tar.gz to ./output.tar.gz


In [19]:
#Download the model metrics
s3 = boto3.resource('s3')

s3.Bucket(bucket).download_file(job_key, './output.tar.gz')

In [20]:
#Unpack the gzip file
!tar xvzf ./output.tar.gz

output/
output/confusion_matrix.json


In [21]:
import json

with open('./output/confusion_matrix.json') as json_file:
    data = json.load(json_file)
print(json.dumps(data, indent=2, default=str))

{
  "confusion_matrix": [
    [
      396,
      146,
      48,
      12,
      18
    ],
    [
      150,
      253,
      146,
      49,
      22
    ],
    [
      53,
      164,
      272,
      107,
      24
    ],
    [
      15,
      33,
      121,
      258,
      193
    ],
    [
      10,
      18,
      27,
      109,
      456
    ]
  ],
  "labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ],
  "type": "multi_class",
  "all_labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ]
}


In [22]:
!pip install tabulate

boto3 1.12.39 has requirement botocore<1.16.0,>=1.15.39, but you'll have botocore 1.16.16 which is incompatible.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [23]:
from IPython.display import HTML, display
import tabulate
table = [['', '1', '2', '3', '4', '5', '(Predicted)'],
         ['1', data['confusion_matrix'][0][0], data['confusion_matrix'][0][1], data['confusion_matrix'][0][2], data['confusion_matrix'][0][3], data['confusion_matrix'][0][4]],
         ['2', data['confusion_matrix'][1][0], data['confusion_matrix'][1][1], data['confusion_matrix'][1][2], data['confusion_matrix'][1][3], data['confusion_matrix'][1][4]],
         ['3', data['confusion_matrix'][2][0], data['confusion_matrix'][2][1], data['confusion_matrix'][2][2], data['confusion_matrix'][2][3], data['confusion_matrix'][2][4]],
         ['4', data['confusion_matrix'][3][0], data['confusion_matrix'][3][1], data['confusion_matrix'][3][2], data['confusion_matrix'][3][3], data['confusion_matrix'][3][4]],
         ['5', data['confusion_matrix'][4][0], data['confusion_matrix'][4][1], data['confusion_matrix'][4][2], data['confusion_matrix'][4][3], data['confusion_matrix'][4][4]],
         ['(Actual)']]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

,1,2,3,4,5,(Predicted)
1,396,146,48,12,18,
2,150,253,146,49,22,
3,53,164,272,107,24,
4,15,33,121,258,193,
5,10,18,27,109,456,
(Actual),,,,,,


# Deploy Endpoint

In [24]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

comprehend_endpoint_name = 'comprehend-inference-ep-' + timestamp_suffix

inference_endpoint_response = comprehend.create_endpoint(
    EndpointName=comprehend_endpoint_name,
    ModelArn = model_arn,
    DesiredInferenceUnits = 1
)

In [25]:
comprehend_endpoint_arn = inference_endpoint_response["EndpointArn"]
print(comprehend_endpoint_arn)

arn:aws:comprehend:us-west-2:853609162619:document-classifier-endpoint/comprehend-inference-ep-23-21-51-47


# Pass Variables to the Next Notebook(s)

In [26]:
%store comprehend_endpoint_arn

Stored 'comprehend_endpoint_arn' (str)


# Predict with Endpoint

In [27]:
describe_response = comprehend.describe_endpoint(
    EndpointArn = comprehend_endpoint_arn
)
print(describe_response)

{'EndpointProperties': {'EndpointArn': 'arn:aws:comprehend:us-west-2:853609162619:document-classifier-endpoint/comprehend-inference-ep-23-21-51-47', 'Status': 'CREATING', 'ModelArn': 'arn:aws:comprehend:us-west-2:853609162619:document-classifier/Amazon-Customer-Reviews-Classifier-1590268028', 'DesiredInferenceUnits': 1, 'CurrentInferenceUnits': 0, 'CreationTime': datetime.datetime(2020, 5, 23, 21, 51, 47, 96000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 5, 23, 21, 51, 47, 96000, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': 'ec0b0cb2-5d80-454d-8bf4-99cda22d55aa', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ec0b0cb2-5d80-454d-8bf4-99cda22d55aa', 'content-type': 'application/x-amz-json-1.1', 'content-length': '408', 'date': 'Sat, 23 May 2020 21:51:46 GMT'}, 'RetryAttempts': 0}}


In [28]:
import time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_response = comprehend.describe_endpoint(
        EndpointArn = comprehend_endpoint_arn
    )
    status = describe_response["EndpointProperties"]["Status"]
    print("Endpoint: {}".format(status))
    
    if status == "IN_SERVICE" or status == "IN_ERROR":
        break
        
    time.sleep(5)

Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CR

In [29]:
txt = """I loved it!  I will recommend this to everyone."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "5",
      "Score": 0.9423999786376953
    },
    {
      "Name": "4",
      "Score": 0.034699998795986176
    },
    {
      "Name": "1",
      "Score": 0.00989999994635582
    }
  ],
  "ResponseMetadata": {
    "RequestId": "dc39a87c-f7bb-439b-b35a-2307bf2d7f7f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "dc39a87c-f7bb-439b-b35a-2307bf2d7f7f",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "136",
      "date": "Sat, 23 May 2020 22:01:09 GMT"
    },
    "RetryAttempts": 0
  }
}


In [30]:
txt = """It's OK."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "3",
      "Score": 0.6840999722480774
    },
    {
      "Name": "4",
      "Score": 0.12229999899864197
    },
    {
      "Name": "2",
      "Score": 0.11890000104904175
    }
  ],
  "ResponseMetadata": {
    "RequestId": "48653fc8-2d6a-48c9-99b7-6581347c76b4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "48653fc8-2d6a-48c9-99b7-6581347c76b4",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "135",
      "date": "Sat, 23 May 2020 22:01:09 GMT"
    },
    "RetryAttempts": 0
  }
}


In [31]:
txt = """Really bad.  I hope they don't make this anymore."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "1",
      "Score": 0.49149999022483826
    },
    {
      "Name": "2",
      "Score": 0.22550000250339508
    },
    {
      "Name": "3",
      "Score": 0.22050000727176666
    }
  ],
  "ResponseMetadata": {
    "RequestId": "38002e6a-018b-42f6-9bbb-1ec8ad3bd8dc",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "38002e6a-018b-42f6-9bbb-1ec8ad3bd8dc",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "136",
      "date": "Sat, 23 May 2020 22:01:09 GMT"
    },
    "RetryAttempts": 0
  }
}


# Navigate to Comprehend in the AWS Console
![Comprehend Console](img/comprehend-console.png)